In [21]:
import os

import gym
import pandas as pd
import numpy as np
import torch as th
import pickle
from collections import defaultdict

from utils.plotting import plot_costs
from apmd_off.iapmd import IPMD
from stable_baselines3.common.env_util import make_vec_env

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
instances = ['Ant', 'HalfCheetah', 'Hopper', 'Humanoid', 'Walker2d']

### SPMD

In [60]:
# Ant

folder = '/Users/patrick/Developer/Inverse-Policy-Mirror-Descent/logs/apmd/Ant-v4-2023-01-14 09:21:07'
file = np.load(folder + '/evaluations.npz')
rewards = file['results']
mean = rewards.mean(axis=1)
std = rewards.std(axis=1)
best = mean.max()
idx = np.argmax(mean)
print(len(mean)*500)
print(f'{mean[idx]:.2f} +- {std[idx]:.2f}')

3000000
5230.03 +- 117.82


In [61]:
# Halfcheetah

folder = '/Users/patrick/Developer/Inverse-Policy-Mirror-Descent/logs/apmd/sac-style/HalfCheetah-v4-2022-11-28 23:19:30'
file = np.load(folder + '/evaluations.npz')
rewards = file['results']
mean = rewards.mean(axis=1)
std = rewards.std(axis=1)
best = mean.max()
idx = np.argmax(mean)
print(len(mean)*500)
print(f'{mean[idx]:.2f} +- {std[idx]:.2f}')

3000000
13025.10 +- 37.74


In [62]:
# Hopper

folder = '/Users/patrick/Developer/Inverse-Policy-Mirror-Descent/logs/apmd/sac-style/Hopper-v4-2023-01-12 23:22:24'
file = np.load(folder + '/evaluations.npz')
rewards = file['results']
mean = rewards.mean(axis=1)
std = rewards.std(axis=1)
best = mean.max()
idx = np.argmax(mean)
print(len(mean)*500)
print(f'{mean[idx]:.2f} +- {std[idx]:.2f}')

3000000
3618.92 +- 5.29


In [65]:
# Humanoid

folder = '/Users/patrick/Developer/Inverse-Policy-Mirror-Descent/logs/apmd/sac-style/Humanoid-v4-2022-12-01 10:22:13'
file = np.load(folder + '/evaluations.npz')
rewards = file['results'][:6000]
mean = rewards.mean(axis=1)
std = rewards.std(axis=1)
best = mean.max()
idx = np.argmax(mean)
print(len(mean)*500)
print(f'{mean[idx]:.2f} +- {std[idx]:.2f}')

3000000
10249.44 +- 7.31


In [66]:
# Walker2d

folder = '/Users/patrick/Developer/Inverse-Policy-Mirror-Descent/logs/apmd/sac-style/Walker2d-v4-2023-01-02 12:46:58'
file = np.load(folder + '/evaluations.npz')
rewards = file['results'][:6000]
mean = rewards.mean(axis=1)
std = rewards.std(axis=1)
best = mean.max()
idx = np.argmax(mean)
print(len(mean)*500)
print(f'{mean[idx]:.2f} +- {std[idx]:.2f}')

3000000
4453.65 +- 26.24


### SPMD IRL

In [67]:
# Ant

folder = 'logs/iapmd/10traj/Ant-v4-2022-10-23 18:07:52'
file = np.load(folder + '/evaluations.npz')
rewards = file['results']
mean = rewards.mean(axis=1)
std = rewards.std(axis=1)
best = mean.max()
idx = np.argmax(mean)
print(len(mean)*500)
print(f'{mean[idx]:.2f} +- {std[idx]:.2f}')

2000000
4053.22 +- 110.13


In [68]:
# HalfCheetah

folder = 'logs/iapmd/10traj/HalfCheetah-v4-2022-10-24 08:42:41'
file = np.load(folder + '/evaluations.npz')
rewards = file['results']
mean = rewards.mean(axis=1)
std = rewards.std(axis=1)
best = mean.max()
idx = np.argmax(mean)
print(len(mean)*500)
print(f'{mean[idx]:.2f} +- {std[idx]:.2f}')

2000000
12633.68 +- 525.25


In [69]:
# Hopper

folder = 'logs/iapmd/10traj/Hopper-v4-2022-10-28 14:40:03'
file = np.load(folder + '/evaluations.npz')
rewards = file['results']
mean = rewards.mean(axis=1)
std = rewards.std(axis=1)
best = mean.max()
idx = np.argmax(mean)
print(len(mean)*500)
print(f'{mean[idx]:.2f} +- {std[idx]:.2f}')

2000000
3563.69 +- 15.57


In [73]:
# Humanoid

folder = 'logs/iapmd/10traj/Humanoid-v4-2023-01-13 00:37:54'
file = np.load(folder + '/evaluations.npz')
rewards = file['results'][:4000]
mean = rewards.mean(axis=1)
std = rewards.std(axis=1)
best = mean.max()
idx = np.argmax(mean)
print(len(mean)*500)
print(f'{mean[idx]:.2f} +- {std[idx]:.2f}')

2000000
7379.37 +- 46.35


In [71]:
# Walker2d

folder = 'logs/iapmd/10traj/Walker2d-v4-2022-10-25 00:04:15'
file = np.load(folder + '/evaluations.npz')
rewards = file['results']
mean = rewards.mean(axis=1)
std = rewards.std(axis=1)
best = mean.max()
idx = np.argmax(mean)
print(len(mean)*500)
print(f'{mean[idx]:.2f} +- {std[idx]:.2f}')

2000000
5423.29 +- 7.30


### Compare

In [36]:
rlbestmodel = {}
rlbestmodel['Ant']='/Users/patrick/Developer/Inverse-Policy-Mirror-Descent/logs/apmd/Ant-v4-2023-01-14 09:21:07'
rlbestmodel['HalfCheetah']='/Users/patrick/Developer/Inverse-Policy-Mirror-Descent/logs/apmd/sac-style/HalfCheetah-v4-2022-11-28 23:19:30'
rlbestmodel['Hopper']='/Users/patrick/Developer/Inverse-Policy-Mirror-Descent/logs/apmd/sac-style/Hopper-v4-2023-01-12 23:22:24'
rlbestmodel['Humanoid']='/Users/patrick/Developer/Inverse-Policy-Mirror-Descent/logs/apmd/sac-style/Humanoid-v4-2022-12-01 10:22:13'
rlbestmodel['Walker2d']='/Users/patrick/Developer/Inverse-Policy-Mirror-Descent/logs/apmd/sac-style/Walker2d-v4-2023-01-02 12:46:58'

In [37]:
irlbestmodel = {}
irlbestmodel['Ant']='logs/iapmd/10traj/Ant-v4-2022-10-23 18:07:52'
irlbestmodel['HalfCheetah']='logs/iapmd/10traj/HalfCheetah-v4-2022-10-24 08:42:41'
irlbestmodel['Hopper']='logs/iapmd/10traj/Hopper-v4-2022-10-28 14:40:03'
irlbestmodel['Humanoid']='logs/iapmd/10traj/Humanoid-v4-2023-01-13 00:37:54'
irlbestmodel['Walker2d']='logs/iapmd/10traj/Walker2d-v4-2022-10-25 00:04:15'


#### RL

#### IRL

In [6]:
# Ant

folder = '/Users/patrick/Developer/Inverse-Policy-Mirror-Descent/logs/sac/0.99/Ant-v4-2023-01-04 04:19:02'
file = np.load(folder + '/evaluations.npz')
rewards = file['results']
mean = rewards.mean(axis=1)
std = rewards.std(axis=1)
best = mean.max()
idx = np.argmax(mean)
print(len(mean)*500)
print(f'{mean[idx]:.2f} +- {std[idx]:.2f}')

3000000
5117.58 +- 56.53


In [7]:
# Halfcheetah

folder = '/Users/patrick/Developer/Inverse-Policy-Mirror-Descent/logs/sac/0.99/HalfCheetah-v4-2023-01-03 16:20:21'
file = np.load(folder + '/evaluations.npz')
rewards = file['results']
mean = rewards.mean(axis=1)
std = rewards.std(axis=1)
best = mean.max()
idx = np.argmax(mean)
print(len(mean)*500)
print(f'{mean[idx]:.2f} +- {std[idx]:.2f}')

3000000
13300.18 +- 45.85


In [8]:
# Hopper

folder = '/Users/patrick/Developer/Inverse-Policy-Mirror-Descent/logs/sac/0.99/Hopper-v4-2023-01-03 10:42:37'
file = np.load(folder + '/evaluations.npz')
rewards = file['results']
mean = rewards.mean(axis=1)
std = rewards.std(axis=1)
best = mean.max()
idx = np.argmax(mean)
print(len(mean)*500)
print(f'{mean[idx]:.2f} +- {std[idx]:.2f}')

3000000
3875.68 +- 78.72


In [9]:
# Humanoid

folder = '/Users/patrick/Developer/Inverse-Policy-Mirror-Descent/logs/sac/0.99/Humanoid-v4-2023-01-04 11:32:57'
file = np.load(folder + '/evaluations.npz')
rewards = file['results'][:6000]
mean = rewards.mean(axis=1)
std = rewards.std(axis=1)
best = mean.max()
idx = np.argmax(mean)
print(len(mean)*500)
print(f'{mean[idx]:.2f} +- {std[idx]:.2f}')

3000000
6922.67 +- 10.68


In [10]:
# Walker2d

folder = '/Users/patrick/Developer/Inverse-Policy-Mirror-Descent/logs/sac/0.99/Walker2d-v4-2023-01-03 22:28:29'
file = np.load(folder + '/evaluations.npz')
rewards = file['results'][:6000]
mean = rewards.mean(axis=1)
std = rewards.std(axis=1)
best = mean.max()
idx = np.argmax(mean)
print(len(mean)*500)
print(f'{mean[idx]:.2f} +- {std[idx]:.2f}')

3000000
6115.45 +- 40.80


### Reward recovery for Pendulum and LunarLanderContinuous

In [97]:
def totorch(a:np.array):
    return th.from_numpy(np.array([a]))

def get_reward(reward_func, state, action):
    state = totorch(state)
    action = totorch(action)
    return reward_func(state, action)[0].detach()[0,0].numpy()

#### Pendulum

In [123]:
pen_env = make_vec_env('Pendulum-v1')
pen_env.reset()
pen_bestmodelpath = '/Users/patrick/Developer/Inverse-Policy-Mirror-Descent/logs/iapmd/reward_recovery/Pendulum-v1-2023-01-23 13:07:11/best_model.zip'
pen_model = IPMD('MlpPolicy', pen_env, expert_replay_buffer_loc='/Users/patrick/Developer/Inverse-Policy-Mirror-Descent/reward_recovery/Pendulum-v1/best_model.pkl')
pen_model.set_parameters(pen_bestmodelpath)
pen_model_reward = pen_model.policy.reward_est

tensor(-126.7933)


In [188]:
# pen_env.observation_space.sample()
pen_env.action_space.shape
state = pen_env.reset()
sample_size = 2000
reward_record = []
rewards = []
predicted_rewards = []
for i in range(sample_size):
    action, _ = pen_model.predict(state)
    predicted_reward = get_reward(pen_model_reward, state[0], action[0])
    new_state, reward, done, _ = pen_env.step(action)
    reward_record.append([state[0], action[0], float(reward[0]), float(predicted_reward)])
    state = new_state
    rewards.append(float(reward[0]))
    predicted_rewards.append(float(predicted_reward))
    if done:
        state = pen_env.reset()
#     print(reward, predicted_reward)
rewards = np.array(rewards)
predicted_rewards = np.array(predicted_rewards)
spnorm = np.max(rewards - predicted_rewards) - np.min(rewards - predicted_rewards)
print(spnorm)
print(rewards.sum(), predicted_rewards.sum())

15.487541276961565
-3359.2796535986563 42.9754421338439


In [139]:
centered_rewards = rewards - np.mean(rewards)
centered_prediction = predicted_rewards - np.mean(predicted_rewards)

In [140]:
centered_rewards - centered_prediction

array([-2.59332938e+00, -2.62558506e+00, -2.61331228e+00, -2.81361015e+00,
       -3.15191005e+00, -3.55577604e+00, -4.01531790e+00, -4.74538691e+00,
       -5.51334465e+00, -5.31418867e+00, -4.70994217e+00, -3.83329319e+00,
       -3.36737291e+00, -2.60469640e+00, -2.15228448e+00, -1.59480513e+00,
       -1.32916591e+00, -9.85309699e-01, -9.54297223e-01, -9.86733534e-01,
       -1.19680492e+00, -1.35446768e+00, -2.16794691e+00, -2.65819911e+00,
       -3.60999791e+00, -4.53123240e+00, -5.59271208e+00, -5.86348657e+00,
       -4.89334313e+00, -4.13886229e+00, -3.35291273e+00, -2.56303016e+00,
       -1.92440290e+00, -1.22642133e+00, -6.44427755e-01, -2.68084803e-01,
       -8.82442679e-02, -1.17616990e-01, -3.73990872e-01, -8.74726751e-01,
       -1.39570985e+00, -2.07207421e+00, -2.96756188e+00, -3.72530580e+00,
       -4.66261993e+00, -5.42789964e+00, -5.90003628e+00, -5.02865089e+00,
       -4.12747900e+00, -3.31230644e+00, -2.22310642e+00, -1.43298213e+00,
       -9.50231859e-01, -

In [143]:
print(rewards.sum(), predicted_rewards.sum())

-2518.678815489271 16.022188492119312


In [190]:
with open('reward_record', 'wb') as fp:
    pickle.dump(reward_record, fp, protocol=4)

In [191]:
print(reward_record[0])

[array([-0.38244265,  0.9239792 , -0.8220529 ], dtype=float32), array([-0.08442295], dtype=float32), -3.921875, -0.007520027458667755]


#### LunarLander

In [200]:
luna_env = make_vec_env('LunarLanderContinuous-v2')
luna_env.reset()
luna_bestmodelpath = '/Users/patrick/Developer/Inverse-Policy-Mirror-Descent/logs/iapmd/reward_recovery/LunarLanderContinuous-v2-2023-01-25 10:08:07/best_model.zip'
luna_model = IPMD('MlpPolicy', 
                  luna_env, 
                  traj_size=275,
                  expert_replay_buffer_loc='/Users/patrick/Developer/Inverse-Policy-Mirror-Descent/reward_recovery/LunarLanderContinuous-v2/best_model.pkl')
luna_model.set_parameters(luna_bestmodelpath)
luna_model_reward = luna_model.policy.reward_est


tensor(159.0948)


In [204]:
# pen_env.observation_space.sample()
luna_env.action_space.shape
sample_size = 10000
reward_records = []
for j in range(10):
    state = luna_env.reset()
    reward_record = []
    rewards = []
    predicted_rewards = []
    for i in range(sample_size):
        action, _ = luna_model.predict(state)
        predicted_reward = get_reward(luna_model_reward, state[0], action[0])
        new_state, reward, done, _ = luna_env.step(action)
        reward_record.append([state[0], action[0], float(reward[0]), float(predicted_reward)])
        state = new_state
        rewards.append(float(reward[0]))
        predicted_rewards.append(float(predicted_reward))
        if done:
            state = pen_env.reset()
            break
    #     print(reward, predicted_reward)
    rewards = np.array(rewards)
    predicted_rewards = np.array(predicted_rewards)
    spnorm = np.max(rewards - predicted_rewards) - np.min(rewards - predicted_rewards)
    print(spnorm)
    print(rewards.sum(), predicted_rewards.sum())
    reward_records.append(reward_record)
    

109.77014741115272
-79.86249126493931 -0.06179475039243698
108.1604351811111
-150.57487260107882 -0.06298962607979774
112.75528492312878
-60.43973008566536 -0.05665504187345505
174.06136233359575
3.472524419426918 -0.044428170658648014
106.36302272975445
-226.54292697273195 -0.040388716384768486
102.40382558852434
-448.99108516052365 -0.0508731622248888
114.680579373613
-95.77025914564729 0.022300509735941887
119.0490738209337
-107.98444949835539 -0.0589103065431118
106.02006690949202
-245.39729543967405 -0.03366057947278023
189.94455938972533
-111.55379484733567 -0.08880027011036873


In [206]:
with open('luna_reward_records', 'wb') as fp:
    pickle.dump(reward_records, fp, protocol=4)